# Compare Computer vison technique

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from skimage import data, filters, segmentation, morphology, measure
from skimage.morphology import disk
from scipy import ndimage as ndi

# Configuration de la figure
plt.figure(figsize=(18, 12))
plt.suptitle('Comparaison des techniques de segmentation simple', fontsize=16, y=1.02)

# 1. Charger et afficher l'image originale
image = data.coffee()
gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)

plt.subplot(2, 3, 1)
plt.imshow(image)
plt.title('1. Image originale')
plt.axis('off')

# 2. Seuillage d'Otsu
thresh = filters.threshold_otsu(gray)
binary = gray > thresh

plt.subplot(2, 3, 2)
plt.imshow(binary, cmap='gray')
plt.title('2. Seuillage Otsu')
plt.axis('off')

# 3. Détection de contours (Sobel)
edges = filters.sobel(gray)

plt.subplot(2, 3, 3)
plt.imshow(edges, cmap='gray')
plt.title('3. Détection de contours (Sobel)')
plt.axis('off')

# 4. Segmentation Watershed
distance = ndi.distance_transform_edt(binary)
local_max = filters.maximum_filter(distance, size=20) == distance
markers = measure.label(local_max)
segmentation_ws = morphology.watershed(-distance, markers, mask=binary)

plt.subplot(2, 3, 4)
plt.imshow(segmentation_ws, cmap='nipy_spectral')
plt.title('4. Segmentation Watershed')
plt.axis('off')

# 5. Segmentation K-means
pixels = image.reshape((-1, 3))
pixels = np.float32(pixels)
k = 3
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 100, 0.2)
_, labels, centers = cv2.kmeans(pixels, k, None, criteria, 10, cv2.KMEANS_RANDOM_CENTERS)
segmented = centers[labels.flatten()].reshape(image.shape)

plt.subplot(2, 3, 5)
plt.imshow(np.uint8(segmented))
plt.title('5. Segmentation K-means (k=3)')
plt.axis('off')

# 6. Opérations morphologiques (dilation)
selem = disk(5)
dilated = morphology.dilation(binary, selem)

plt.subplot(2, 3, 6)
plt.imshow(dilated, cmap='gray')
plt.title('6. Morphologie (Dilation)')
plt.axis('off')

plt.tight_layout()
plt.show()